In [1]:
%run common_imports.py
import spyglass.position.v1 as sgp
from spyglass.position import PositionOutput
import spyglass.linearization.v1 as sgpl
from track_linearization import *

dj.config['display.limit'] = 10**3
%matplotlib qt
plt.style.use(["seaborn-v0_8-talk"])

%load_ext autoreload
%autoreload 2

[2025-12-26 12:14:52,414][INFO]: Connecting anirudh@172.16.102.154:3306
[2025-12-26 12:14:52,429][INFO]: Connected anirudh@172.16.102.154:3306


<!-- ## Get data, make trackgraph -->

In [ ]:
# {'track_graph': <networkx.classes.graph.Graph object at 0x7d94c0565f90>, 'node_positions': [(np.float64(124.58552024980023), np.float64(105.99904868607955)), 
#                                                                                             (np.float64(7.357738431618428), np.float64(107.96595777698863)), 
#                                                                                             (np.float64(6.046465704345703), np.float64(58.268721413352274)),
#                                                                                             (np.float64(124.45439297707297), np.float64(57.35083050426136)), 
#                                                                                             (np.float64(5.653083886163886), np.float64(6.473448686079553)), 
#                                                                                             (np.float64(124.45439297707297), np.float64(5.9489395951704465))],
#  'edges': [(0, 1), (1, 2), (2, 3), (2, 4), (4, 5)],

In [8]:
#define trackgraph
node_positions = [(np.float64(145.830481040261), np.float64(120.08972088068182)),
                  (np.float64(9.255935585715555), np.float64(121.77772088068183)),
                  (np.float64(8.181753767533737), np.float64(64.84608451704545)),
                  (np.float64(145.67702649480646), np.float64(65.15299360795454)), 
                  (np.float64(7.107571949351922), np.float64(6.379902698863649)),
                  (np.float64(145.67702649480646), np.float64(4.998811789772725))]



edges = np.array([(0, 1),
                  (1, 2),
                  (2, 3),
                  (2, 4), 
                  (4, 5)])


wtrack_edge_order = [
        (0, 1),
        (1, 2),
        (2, 3),
        (2, 4),
        (4, 5),
    ]

# wtrack_edge_spacing = [15, 0, 15, 0]
wtrack_edge_spacing = 0

In [9]:
sgpl.TrackGraph.insert1(
    {
        "track_graph_name": "Wtrack_wilbur20210512",
        "environment": "Wtrack",
        "node_positions": node_positions,
        "edges": edges,
        "linear_edge_order": wtrack_edge_order,
        "linear_edge_spacing": wtrack_edge_spacing,
    },
    skip_duplicates=True,
)

graph = sgpl.TrackGraph & {"track_graph_name": "Wtrack_wilbur20210512"}


In [15]:
pos_key = {
    "nwb_file_name": nwb_copy_file_name,
    "epoch": "2", 
    "dlc_model_params_name": "WtrackSep5",
    "dlc_centroid_params_name": "four_paw_centroid"  
}

merge_id = (PositionOutput.DLCPosV1() & pos_key).fetch1("merge_id")
centroid_position = (PositionOutput & {"merge_id": merge_id}).fetch1_dataframe()

In [16]:

with plt.style.context("tableau-colorblind10"):
    fig, ax = plt.subplots()
    graph.plot_track_graph(draw_edge_labels=True)
    for lines in ax.lines:
        lines.set_color("black")
        ax.tick_params(which='both',
                   bottom=True, left=True,
                   labelbottom=True, labelleft=True,
                   colors="white")
        ax.set_xlabel("x_position (cm)")
        ax.set_ylabel("y_position (cm)")


In [17]:
graph

track_graph_name,environment Type of Environment,"node_positions 2D position of nodes, (n_nodes, 2)","edges shape (n_edges, 2)","linear_edge_order order of edges in linear space, (n_edges, 2)","linear_edge_spacing space btwn edges in linear space, (n_edges,)",edge_map Maps one edge to another before linearization
Wtrack_wilbur20210512,Wtrack,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=


In [18]:

with plt.style.context("bmh"):
    ax = centroid_position.plot.scatter(x = "position_x", 
                                  y = "position_y", 
                                  s= 2,
                                  c = "speed",
                                  vmin = 0,
                                  vmax = 80, 
                                  figsize = [10, 10], 
                                  layout = "tight",
                                  xlabel = "position_x (cm)",
                                  ylabel = "position_y (cm)",
                                  colorbar = False
                                  )
    cbar = plt.colorbar(ax.collections[0], ax=ax)
    cbar.set_label("Speed (cm/s)")
    
    graph.plot_track_graph(ax=ax, draw_edge_labels=True)

    ax.tick_params(which='both',
                   bottom=True, left=True,
                   labelbottom=True, labelleft=True,
                   colors="black")
    for ln in ax.lines:
        ln.set_color("black")  

## linearize

In [9]:
sgpl.LinearizationParameters.insert1(
    {"linearization_param_name": "default"}, skip_duplicates=True
)
sgpl.LinearizationParameters()

linearization_param_name name for this set of parameters,use_hmm use HMM to determine linearization,route_euclidean_distance_scaling,sensor_std_dev Uncertainty of position sensor (in cm).,diagonal_bias
default,0,1.0,5.0,0.5
use_hmm,1,1.0,5.0,0.5


In [3]:
from spyglass.linearization.merge import LinearizedPositionOutput


In [24]:
#loop over epochs
epochs = [4, 6, 8]
for epoch in epochs:

    pos_key = {
        "nwb_file_name": nwb_copy_file_name,
        "epoch": f"{epoch}", 
        "dlc_model_params_name": "WtrackSep5",
        "dlc_centroid_params_name": "four_paw_centroid"  
    }

    merge_id = (PositionOutput.DLCPosV1() & pos_key).fetch1("merge_id")
    # nose_position = (PositionOutput & {"merge_id": merge_id}).fetch1_dataframe()
    
    
    sgpl.LinearizationSelection.insert1(
        {
            "pos_merge_id": merge_id,
            "track_graph_name": "Wtrack_wilbur20210512",
            "linearization_param_name": "default",
        },
        skip_duplicates=True,
    )

    sgpl.LinearizationSelection() 
    
    sgpl.LinearizedPositionV1().populate()
    sgpl.LinearizedPositionV1()
    
    linear_key = {
        "pos_merge_id": merge_id,
        "track_graph_name": "Wtrack_wilbur20210512",
        "linearization_param_name": "default",
    }


    linear_merge_key = LinearizedPositionOutput.merge_restrict(linear_key).fetch1(
        "KEY"
    )
    linear_position_df = (
        LinearizedPositionOutput & linear_merge_key
    ).fetch1_dataframe()



[11:58:51][INFO] Spyglass: Computing linear position for: {'pos_merge_id': UUID('9011ad40-a972-3570-9e37-4eff9486ac30'), 'track_graph_name': 'Wtrack_wilbur20210512', 'linearization_param_name': 'default'}
[11:59:43][INFO] Spyglass: Writing new NWB file wilbur20210512_EJ74F55V02.nwb
[12:00:33][INFO] Spyglass: Computing linear position for: {'pos_merge_id': UUID('fbf3854e-a0a2-e83e-8019-16353ac4731d'), 'track_graph_name': 'Wtrack_wilbur20210512', 'linearization_param_name': 'default'}
[12:01:24][INFO] Spyglass: Writing new NWB file wilbur20210512_2Y7C0YWF92.nwb
[12:02:15][INFO] Spyglass: Computing linear position for: {'pos_merge_id': UUID('3f0b9318-0b73-46f3-bb15-9bdd8e47fce7'), 'track_graph_name': 'Wtrack_wilbur20210512', 'linearization_param_name': 'default'}
[12:03:07][INFO] Spyglass: Writing new NWB file wilbur20210512_680ISW3Q3D.nwb


In [25]:
linear_key = {
    "pos_merge_id": merge_id,
    "track_graph_name": "Wtrack_wilbur20210512",
    "linearization_param_name": "default",
}


linear_merge_key = LinearizedPositionOutput.merge_restrict(linear_key).fetch1(
    "KEY"
)
linear_position_df = (
    LinearizedPositionOutput & linear_merge_key
).fetch1_dataframe()

In [13]:
fig, ax = plt.subplots()
ax.scatter(linear_position_df.index.values, linear_position_df["linear_position"])

In [19]:
from spyglass.position.position_merge import PositionOutput
epoch = 8
pos_key = {
    "nwb_file_name": nwb_copy_file_name,
    "epoch": f"{epoch}", 
    "dlc_model_params_name": "WtrackSep5",
    "dlc_centroid_params_name": "four_paw_centroid"  
}

merge_id = (PositionOutput.DLCPosV1() & pos_key).fetch1("merge_id")
position_xy = (PositionOutput() & {"merge_id": merge_id}).fetch1_dataframe()


linear_key = {
    "pos_merge_id": merge_id,
    "track_graph_name": "Wtrack_wilbur20210512",
    "linearization_param_name": "default",
}


linear_merge_key = LinearizedPositionOutput.merge_restrict(linear_key).fetch1(
    "KEY"
)
linear_position_df = (
        LinearizedPositionOutput & linear_merge_key
    ).fetch1_dataframe()

In [44]:
position_xy.plot(x = "position_x", y = "position_y")

<Axes: xlabel='position_x'>

## Plot, animate

In [5]:
concat_position = pd.concat([position_xy, linear_position_df], axis=1)
concat_position = concat_position.rename(columns = {"position_x":"x_position", "position_y": "y_position"})

concat_position = concat_position.copy()

concat_position.index = pd.to_timedelta(concat_position.index, unit="s"
                                        )

In [6]:
clean = concat_position.copy()
clean["linear_position"] = pd.to_numeric(clean["linear_position"], errors="coerce")
clean = clean[np.isfinite(clean["linear_position"].to_numpy())]


In [10]:
make_actual_vs_linearized_position_movie(graph.get_networkx_track_graph(), clean, slice(0, 10000), "test linearization", 125)

In [20]:
fig, ax = plt.subplots(figsize=(20, 13))
ax.scatter(
    linear_position_df.index,
    linear_position_df.linear_position,
    c=linear_position_df.track_segment_id,
    s=1,
    cmap = "viridis"
)
graph.plot_track_graph_as_1D(
    ax=ax, axis="y", other_axis_start=linear_position_df.index[-1] + 10
)

ax.set_xlabel("Time [s]", fontsize=18)
ax.set_ylabel("Linear Position [cm]", fontsize=18)
ax.set_title("Linear Position", fontsize=28)

Text(0.5, 1.0, 'Linear Position')

In [22]:
linear_position_df_s = linear_position_df.iloc[2000:5500]

with plt.style.context("dark_background"):
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.set_xlabel("x-position [cm]", fontsize=18)
    ax.set_ylabel("y-position [cm]", fontsize=18)
    plot = ax.scatter(
        linear_position_df_s.projected_x_position,
        linear_position_df_s.projected_y_position,
        c = linear_position_df_s.index - linear_position_df.index[0],
        cmap = "Blues",
    )
    fig.colorbar(plot, ax = ax, label = "time(s)")
    graph.plot_track_graph(draw_edge_labels=True)
    for lines in ax.lines:
        lines.set_color("white")

    ax.tick_params(which='both',
                   bottom=True, left=True,
                   labelbottom=True, labelleft=True,
                   colors="white")
    # ax.set_xlabel


In [19]:
from matplotlib.animation import FuncAnimation

x = linear_position_df.projected_x_position.to_numpy()
y = linear_position_df.projected_y_position.to_numpy()
t = linear_position_df.index.to_numpy()  # seconds

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ax.set_xlabel("x-position [cm]", fontsize=18)
ax.set_ylabel("y-position [cm]", fontsize=18)
graph.plot_track_graph(ax=ax)  # draw your track graph once

ax.set_xlim(np.nanmin(x) - 5, np.nanmax(x) + 5)
ax.set_ylim(np.nanmin(y) - 5, np.nanmax(y) + 5)


pt,   = ax.plot([], [], 'o', color="#c24444", ms=20, zorder=3)
tail, = ax.plot([], [], 'o', color="#c24444", ms=15, alpha=0.1, zorder=2)
tail_len = int(0.5 * 125)  

def init():
    pt.set_data([], [])
    tail.set_data([], [])
    return pt, tail

def update(i):
    pt.set_data([x[i]], [y[i]])
    j0 = max(0, i - tail_len)
    tail.set_data(x[j0:i+1], y[j0:i+1])
    return pt, tail

target_fps = 125
ani = FuncAnimation(
    fig, update, frames=len(x), init_func=init,
    interval=1000/target_fps, blit=True, repeat=False
)


ani.save('e2_linearised_animation.gif', writer='pillow', fps=125)



(0.0, 100.0)